In [108]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import re
import itertools
from datetime import datetime
pd.options.mode.chained_assignment = None



## TODO create a front end interface to select which tool and contest date
tool = "DK"
# tool = "Yahoo"
# sport = "NHL"
sport = "NBA"
contest_date = "jan26"

path = f"csv/{tool}_{sport}/{contest_date}.csv"


def print_current_df(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(dataframe)

In [109]:
def run_script(tool, sport, date):
#     if tool == 'both':
        #run yahoo normally
            # give choose_relevant_players(df) a cmd paramter which selects all and then exits
            # give choose_relevant_players(df) a healty_players list paramter which records all yahoo players 
        #run DK normally until in choose_relevant players, it uses healthy_players instead of all
            #cmd indicates to use healthy_players instead of all
            
    path = f"csv/{tool}_{sport}/{contest_date}.csv"
    original_df = extract_from_csv(path)
    df = create_df(sport,tool,original_df)
    player_df = choose_relevant_players(df,sport)
    output_sheet(player_df,tool,sport,contest_date)
run_script(tool,sport,contest_date)

Name Of Player:Nikola Jokic, Giannis Antetokounmpo, Luka Doncic, James Harden, Dejounte Murray, Trae Young, Ja Morant, Jimmy Butler, DeMar DeRozan, Pascal Siakam, Fred VanVleet, Jusuf Nurkic, LaMelo Ball, Nikola Vucevic, Kristaps Porzingis, Khris Middleton, Devin Booker, Chris Paul, Miles Bridges, Darius Garland, Bam Adebayo, Julius Randle, Jrue Holiday, Zach LaVine, De'Aaron Fox, Evan Mobley, Tyrese Haliburton, CJ McCollum, Caris LeVert, Jaren Jackson Jr., Jarrett Allen, Anfernee Simons, Tyler Herro, Clint Capela, Terry Rozier, RJ Barrett, Cole Anthony, John Collins, Jakob Poeltl, OG Anunoby, Norman Powell, Derrick White, Wendell Carter Jr., Bobby Portis, Gordon Hayward, Scottie Barnes, Jalen Brunson, Mike Conley, Harrison Barnes, Franz Wagner, Will Barton, Jalen Suggs, Chris Boucher, Hassan Whiteside, Coby White, Bojan Bogdanovic, Reggie Jackson, Aaron Gordon, Ivica Zubac, Jordan Clarkson, Mitchell Robinson, Chris Duarte, Ayo Dosunmu, Monte Morris, Cameron Johnson, Onyeka Okongwu, Om

In [100]:
from IPython.display import display
import ipywidgets as widgets


def sport_selected(arg):
    print("button has been clicked!", arg.description)
    sport = arg.description
    select_tool()

def select_tool():
    button_dk = widgets.Button(description = 'DK')   
    button_yahoo = widgets.Button(description = 'Yahoo')   
    button_dk.on_click(tool_selected)
    button_yahoo.on_click(tool_selected)
    display(button_dk)
    display(button_yahoo)
    
    
def tool_selected(arg):
    print("button has been clicked!", arg.description)
    sport = arg.description
    
    
    
# button_nhl = widgets.Button(description = 'NHL')   
# button_nba = widgets.Button(description = 'NBA')   
# button_nhl.on_click(sport_selected)
# button_nba.on_click(sport_selected)
# display(button_nhl)
# display(button_nba)
# widgets.DatePicker(
#     description='Pick a Date',
#     disabled=False
# )

In [107]:
def extract_from_csv(path):
    original_df = pd.read_csv(path)
    if tool == "DK":
        original_df[['Game','Date','Time','Zone']] = original_df.loc[:,'Game Info'].str.split(" ", 3, expand = True)
        original_df = original_df.rename(columns={'AvgPointsPerGame':'FPPG','TeamAbbrev':'Team'})
    
#     print_current_df(original_df)
    return original_df
original_df = extract_from_csv(path)

KeyError: 'Game Info'

### Modify Original CSV entries to create a DF of relevant data


In [102]:
def convert_positions_to_values(df,original_df):
    
    df.insert(loc=1, column='Pos', value=0)
    conditions = [
        (df['Position'] == 'PG'),
        (df['Position'] == 'SG'),
        (df['Position'] == 'SF'),
        (df['Position'] == 'PF'),
        (df['Position'] == 'C')
    ]
    values = [1, 2, 3, 4, 5]
    df.loc[:,'Pos'] = np.select(conditions, values)

    
def single_game(df,original_df):
    df = original_df[['FPPG','Salary']]
    df.insert(0, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    
def drop_inactive_players(df):
    df = df[df['Injury Status'] != 'INJ'] 
    df = df[df['Injury Status'] != 'O'] 
    df = df[df['Injury Status'] != 'IR'] 
    return df
def create_df(sport,tool,original_df):
    df = pd.DataFrame()
    if tool == "Yahoo":
        df = original_df[['Position', 'Team','Game','Time','Injury Status','FPPG','Salary']].copy()
        df.insert(4, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
        df = drop_inactive_players(df)
        
        if sport == "NHL":
            df.insert(df.columns.get_loc("FPPG"),'Starting',original_df['Starting'])
            df['Value'] = df['FPPG']/df['Salary']
        else:
            number_of_teams = len(original_df['Team'].value_counts())
            if(number_of_teams == 2):
                single_game(df,original_df)
            else:    
                convert_positions_to_values(df,original_df)
                df.head()
    if tool == "DK":
        df = original_df[['Position', 'Name','Team','Game','Time','FPPG','Salary']]
        if sport == "NHL":
            df['Value'] = df['FPPG']/df['Salary']*1000
            df.insert(df.columns.get_loc("FPPG"),'Starting','')
            df.loc[df.Position == 'G','Starting'] = 'No'
    if sport == "NHL":
        df = df.round({'Value': 2})
        df.loc[df['Position'] == 'LW', 'Position'] = 'W'
        df.loc[df['Position'] == 'RW', 'Position'] = 'W'
#     print_current_df(df)
    return df
    
df = create_df(sport,tool,original_df)

### Interface

To be modified as more front end functionality is added

In [103]:
def add_to_lineup(df, player_df, name):
    if player_df['Name'].str.lower().str.contains(name.lower()).any() != True:
        player_df = player_df.append(df.loc[df['Name'].str.lower() == name.lower()])
    else:
        if name != '' : print(name + " is already in list")
    return player_df
def remove_from_lineup(player_df, name):
    print("name? " + name)
    if player_df['Name'].str.lower().str.contains(name.lower()).any() == True:
        player_df.drop(player_df.loc[player_df['Name'].str.lower()==name.lower()].index, inplace=True)
    else:
        if name != ''  : print(name + " is not in in list") 
    return player_df

In [104]:
df2 = df.sort_values(by=['Salary'], ascending = False)
# if sport == 'NHL':
#     df2 = df2.drop(df2[df2.Starting == "No"].index)
df2.drop(df2[df2.FPPG == 0.00].index,inplace=True)
for name in df2['Name']: 
    print(name, end = ", ")


Nikola Jokic, Giannis Antetokounmpo, Luka Doncic, James Harden, Dejounte Murray, Trae Young, Ja Morant, Jimmy Butler, DeMar DeRozan, Pascal Siakam, Fred VanVleet, Jusuf Nurkic, LaMelo Ball, Nikola Vucevic, Kristaps Porzingis, Khris Middleton, Devin Booker, Chris Paul, Miles Bridges, Darius Garland, Bam Adebayo, Julius Randle, Jrue Holiday, Zach LaVine, De'Aaron Fox, Evan Mobley, Tyrese Haliburton, CJ McCollum, Caris LeVert, Jaren Jackson Jr., Jarrett Allen, Anfernee Simons, Tyler Herro, Clint Capela, Terry Rozier, RJ Barrett, Cole Anthony, John Collins, Jakob Poeltl, OG Anunoby, Norman Powell, Derrick White, Wendell Carter Jr., Bobby Portis, Gordon Hayward, Scottie Barnes, Jalen Brunson, Mike Conley, Harrison Barnes, Franz Wagner, Will Barton, Jalen Suggs, Chris Boucher, Hassan Whiteside, Coby White, Bojan Bogdanovic, Reggie Jackson, Aaron Gordon, Ivica Zubac, Jordan Clarkson, Mitchell Robinson, Chris Duarte, Ayo Dosunmu, Monte Morris, Cameron Johnson, Onyeka Okongwu, Omer Yurtseven, D

### Table Add

In [91]:
def get_names(names):
    return [x.strip() for x in names.split(',')]


##adding these blank columns that can be edited in an excel sheet to add the values
##Future versions will let user change the values inside them
##TODO Modify current excel based version to use excel formulas
##https://stackoverflow.com/questions/51348874/save-pandas-dataframes-with-formulas-to-xlsx-files
def add_extra_columns(player_df,sport):
    player_df['Interest'] = ""
    player_df['Floor'] = ""
    player_df['Ceiling'] = ""
    if sport == 'NBA':
        player_df['Estimate'] = ""
    player_df['Actual'] = ""
    if sport == 'NBA':
        player_df['Deviation'] = ""
    player_df['Floor Value'] = ""
    player_df['Ceiling Value'] = ""
    player_df['Border Value'] = ""
    
    if sport == 'NBA':
        player_df['Estimate Value'] = ""
        player_df['Actual Value'] = ""
    player_df['Notes'] = ""
    return player_df
def choose_relevant_players(df, cmd='',sport = 'NBA', tool = 'Yahoo'):
    player_df = pd.DataFrame(columns = df.columns)
    queries = ['exit', 'all', 'show']
    player_df = player_df.replace(np.nan, '', regex=True)
#     if cmd != '': query == 'cmd'
    exit = False
    while exit!=True:
        query = input('Name Of Player:')
        if query not in queries:
            if re.search('^remove', query):
                names_to_remove = re.findall('(?<=remove).*$', query)[0]
                names = get_names(names_to_remove)
                for name in names:
                    name = name.title()
                    player_df = remove_from_lineup(player_df,name)
                    print(f"removed {name}")

            else:
                names = get_names(query)
                for name in names:
                    name = name.title()
                    if df['Name'].str.lower().str.contains(name.lower()).any():
                        player_df = add_to_lineup(df,player_df,name)   
                    else:
                        print(name + " is not in the csv list. Please try again")
        elif query =='show':
            print_current_df(player_df)
        elif query =='all':
            player_df = df
#             print_current_df(player_df)
        else:
#             print_current_df(player_df)
            player_df = add_extra_columns(player_df,sport)
            exit = True
    return player_df


player_df = choose_relevant_players(df)



KeyboardInterrupt: Interrupted by user

In [73]:
def output_sheet(output_df,tool,sport,contest_date):
#     dt = datetime.now()
#     now = dt.strftime("%Y-%m-%d %H_%M")

    file_name = f'output/{tool}/{sport}/{contest_date}_{tool}_{sport}.xlsx'
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined
    # header.

    output_df.to_excel(writer, index = False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Get the dimensions of the dataframe.
    (max_row, max_col) = output_df.shape

    # Create a list of column headers, to use in add_table().exo
    column_settings = []
    for header in player_df.columns:
        column_settings.append({'header': header})

    # Add the table.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)

    if sport == 'NHL': 
        superstar_df = df.loc[df['Salary'] > 18]
        superstar_df.to_excel (writer, index = False, header=True, sheet_name = "Superstars")
        worksheet2 = writer.sheets['Superstars']

        (max_row, max_col) = df.shape
        print_current_df(superstar_df)
        worksheet2.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet2.set_column(0, max_col - 1, 12)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()
    print('succesfully created: ' + file_name)

    
output_sheet(player_df,tool,sport,contest_date)


,Position,Name,Team,Game,Time,Starting,FPPG,Salary,Value
0,C,Aleksander Barkov,FLA,FLA@WPG,08:00PM,,15.19,9000,1.69
1,C,Connor McDavid,EDM,EDM@VAN,10:00PM,,16.92,8600,1.97
2,G,Tristan Jarry,PIT,ARI@PIT,07:00PM,No,16.89,8500,1.99
3,G,Casey DeSmith,PIT,ARI@PIT,07:00PM,No,8.66,8500,1.02
4,G,Ilya Sorokin,NYI,PHI@NYI,07:30PM,No,17.17,8400,2.04
5,G,Semyon Varlamov,NYI,PHI@NYI,07:30PM,No,12.26,8400,1.46
6,G,Louis Domingue,PIT,ARI@PIT,07:00PM,No,33.50,8300,4.04
7,G,Juuse Saros,NSH,NSH@SEA,10:00PM,No,16.90,8300,2.04
8,G,David Rittich,NSH,NSH@SEA,10:00PM,No,8.07,8300,0.97
9,G,Kenneth Appleby,NYI,PHI@NYI,07:30PM,No,0.00,8200,0.00


succesfully created: output/DK/NHL/jan25_DK_NHL.xlsx


### Single-game analysis [WIP]

#### Single Game Entry Strategy:

- **Stars:** Out of the 4-5 likely star players, choose 3.

- **Peasants:** Out of the 5-10 non-star players, choose 5.



In [ ]:
single_df = player_df[['Name','Salary','Floor','Ceiling','Estimate']]
print_current_df(single_df)
create_lineup(single_df)

In [ ]:
max_salary = 30

In [ ]:
## best 5 probable, best 5 worst, best 5 best. Create combinations of lineups with the most recurring names

def create_lineup(df):
    lineup = []
    df2 = df.ge()
        
    
    print(highest_paid_player.Name)
    

In [ ]:
numbers = [1, 2, 3, 7, 7, 9, 10]
result = [seq for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if sum(seq) >= 10]
print(result)

In [ ]:
class Lineup: 
    superstar = ""
    megastar = ""
    star = ""
    other_players = []   
    

In [ ]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self,data):
        self.data = data
l = [1,2,"e"]
x = MyClass(l)
x.data


### Multi-game analysis [WIP]

In [ ]:
# sorted_df = filled_df.sort_values(by=['Pos','Probable'],ascending=[True,False]).drop(['Notes', 'Pts', 'Reb', 'Ast', 'Stl', 'Blk', 'TO',
#        'Total'],axis = 1)

# # sorted_df.insert(12,'Worst Value',sorted_df["Worst"] / sorted_df["Salary"])
# # sorted_df.insert(13,'Best Value',sorted_df["Best"] / sorted_df["Salary"])
# df=sorted_df
# lineups = []
# positions = []



# df = df.loc[:, lambda df: ['Pos','Salary','Name', "Worst","Best",'Probable']]
# [positions.append(df.loc[(sorted_df['Pos'] == i)].sort_values(by=['Pos','Probable'],ascending=[True,False])) for i in range(1,6)]

# [print_current_df(position_df) for position_df in positions]

In [ ]:
# def select_eligible(df):
# new_df = sorted_df.loc[(sorted_df['Salary'] > 30)]
# df

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
